In [1]:
from sklearn.ensemble import GradientBoostingClassifier

# One-Hot Encode categorical variables
train_data_encoded = pd.get_dummies(train_data, columns=['product_code', 'attribute_0', 'attribute_1'])
test_data_encoded = pd.get_dummies(test_data, columns=['product_code', 'attribute_0', 'attribute_1'])

# Ensure the test set has the same columns as the train set
test_data_encoded = test_data_encoded.reindex(columns=train_data_encoded.columns, fill_value=0)

# Impute missing values
cols_to_impute = train_data_encoded.columns[train_data_encoded.isnull().sum() > 0]
train_data_encoded[cols_to_impute] = imputer.fit_transform(train_data_encoded[cols_to_impute])
test_data_encoded[cols_to_impute] = imputer.transform(test_data_encoded[cols_to_impute])

# Separate features and target variable from the training data
X = train_data_encoded.drop(columns=['id', 'failure'])
y = train_data_encoded['failure']

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.transform(test_data_encoded.drop(columns=['id', 'failure']))

# Split the training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Train a Gradient Boosting classifier
model = GradientBoostingClassifier(random_state=42)
model.fit(X_train, y_train)

# Evaluate the model
y_val_pred = model.predict_proba(X_val)[:, 1]
roc_auc = roc_auc_score(y_val, y_val_pred)

# Make predictions on the test data
y_test_pred = model.predict_proba(X_test_scaled)[:, 1]

# Prepare the submission file
submission = pd.DataFrame({'id': test_data['id'], 'failure': y_test_pred})
submission.to_csv('submission_gb.csv', index=False)

roc_auc

NameError: name 'pd' is not defined